## Off the shelf results with T5

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
# Look for gpu to use. Will use 'cpu' by default if no gpu found.
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
local_path = "../"
remote_path = "/notebooks/oreilly-hands-on-transformers/"

#runtime_path = remote_path
runtime_path = local_path

In [ ]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-base')
base_tokenizer = T5Tokenizer.from_pretrained('t5-base')

## Abstractive Summarization

In [ ]:
text_to_summarize ="""Sinan Ozdemir is a data scientist, startup founder, and educator living in the San Francisco Bay Area with his dog, 
Charlie; cat, Euclid; and bearded dragon, Fiero. He spent his academic career studying pure mathematics 
at Johns Hopkins University before transitioning to education. He spent several years conducting lectures 
on data science at Johns Hopkins University and at the General Assembly before founding his own startup, 
Kylie.ai, which uses artificial intelligence to build chatbots from historical transcripts. 
After completing a Fellowship at the Y Combinator accelerator, Sinan spent most of his time working on 
his fast-growing company, while creating educational material for data science.
"""

preprocess_text = text_to_summarize.strip().replace("\n","")

print ("original text preprocessed:\n", preprocess_text)

In [ ]:
# known prompt for summarization with T5
t5_prepared_text = "summarize: " + preprocess_text

input_ids = base_tokenizer.encode(t5_prepared_text, return_tensors="pt")

# summmarize 
summary_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    min_length=30,
    max_length=50,
    early_stopping=True
)

output = base_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print (f"Summarized text: \n{output}")

## English -> German Translation

In [ ]:
input_ids = base_tokenizer.encode('translate English to German: Where is the chocolate?', return_tensors='pt')

# translate 
translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print (f"Translated text:\n{output}")


In [ ]:
# pass labels in to calculate loss

input_ids = base_tokenizer('translate English to German: Where is the chocolate?', return_tensors='pt').input_ids
labels = base_tokenizer('Wo ist die Schokolade?', return_tensors='pt').input_ids

loss = base_model(input_ids=input_ids, labels=labels).loss

labels, loss

## CoLA: The Corpus of Linguistic Acceptability
checking for grammatical correctess

In [ ]:
input_ids = base_tokenizer.encode('cola sentence: Where is the chocolate?', return_tensors='pt')

# CoLA 
translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")


In [ ]:
input_ids = base_tokenizer.encode('cola sentence: Where be a chocolates?', return_tensors='pt')

# summmarize 
translate_ids = base_model.generate(
    input_ids,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")

## STSB - Semantic Text Similarity Benchmark
Are two sentences semantically similar

In [ ]:
sentence_one = 'How to fish'
sentence_two = 'Fishing Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# calculate semantic similarity 
translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

In [ ]:
sentence_one = 'How to fish'
sentence_two = 'Hiking Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# calculate semantic similarity
translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

## MNLI - Multi-Genre Natural Language Inference
Whether a premise implies (“entailment”), contradicts (“contradiction”), or neither (“neutral”) a hypothesis.

In [ ]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I am running for mayor', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

In [ ]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I do not really vote', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

In [ ]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I code for a living', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

## Q/A - Question/Answering

In [ ]:
input_ids = base_tokenizer.encode(
    'question: Where does Sinan live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

In [ ]:
input_ids = base_tokenizer.encode(
    'question: Where does Matt live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

In [ ]:
# Sanity check with random prompts

input_ids = base_tokenizer.encode(
    'prompt1: Where does Matt live? prompt2: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

## Fine-tuning T5 for abstractive summarization

In [ ]:
from transformers import pipeline, T5ForConditionalGeneration, TrainingArguments, Trainer, \
                         DataCollatorForSeq2Seq
import pandas as pd
from datasets import Dataset
import random

In [ ]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-small')
base_tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [ ]:
# https://www.kaggle.com/snap/amazon-fine-food-reviews?select=Reviews.csv

reviews = pd.read_csv(runtime_path + 'data/reviews.csv')

# Pre-processing step
# Punctuation is important in grammar and important for complex decoding architectures to know when to stop!
def add_punc(s):
    if s[-1] not in ('.', '!', '?'):
        s = s + '.'
    return s

reviews.dropna(inplace=True)

reviews['Summary'] = reviews['Summary'].map(add_punc)

print(reviews.shape)

reviews.head()

In [ ]:
reviews = reviews[(reviews['Summary'].str.len() < 100) & (reviews['Summary'].str.len() >= 30)]

reviews.shape

In [ ]:
random.seed(0)

reviews_dataset = Dataset.from_pandas(reviews.astype(str).sample(5000))

In [ ]:
# We have a prompt but only as a prefix in the encoder
prefix = "summarize: "

# we will manually add our own labels because unlike GPT, we cannot assume the labels are based on the inputs
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Text"]]
    model_inputs = base_tokenizer(inputs, max_length=1024, truncation=True)

    labels = base_tokenizer(examples["Summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_reviews_dataset = reviews_dataset.map(preprocess_function, batched=True)

In [ ]:
# inspect a single datapoint
tokenized_reviews_dataset['Summary'][0]

In [ ]:
tokenized_reviews_dataset = tokenized_reviews_dataset.train_test_split(test_size=.1)

In [ ]:
# Data collator specifically for generic sequence to sequence tasks
# Use when we are translating one sequence to another like translation, summarization, etc
data_collator = DataCollatorForSeq2Seq(tokenizer=base_tokenizer, model=base_model)

In [ ]:
training_args = TrainingArguments(
    output_dir=runtime_path + "t5_summary_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
    logging_steps=50,
    save_strategy='epoch'
)

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_reviews_dataset["train"],
    eval_dataset=tokenized_reviews_dataset["test"],
    data_collator=data_collator,
)

trainer.evaluate()

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()

In [ ]:
loaded_model = T5ForConditionalGeneration.from_pretrained(runtime_path + 't5_summary_results')

# summarization pipeline prepends a default prefix of summarize: 
generator = pipeline(
    'summarization', model=loaded_model, tokenizer=base_tokenizer
)

In [ ]:
sam = reviews.sample(1)

print(sam['Summary'])

text = sam['Text'].tolist()[0]
text

In [ ]:
# Generate a summary
generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

In [ ]:
# Try the base t5 on the same text
base_generator = pipeline(
    'summarization', model='t5-small', tokenizer='t5-small'
)

In [ ]:
# Summary is a bit more extractive than our fine-tuned version and style isn't quite the same as our dataset
base_generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)